In [10]:
from transformer_class import SimilarityModel
from data_utils import PhraseDataset
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
import torch
bert_pretrained = 'bert-base-uncased'

# Set up DataLoaders

In [11]:
train_loader = DataLoader(PhraseDataset('processed/train.csv'), batch_size=32, shuffle=True)
val_loader = DataLoader(PhraseDataset('processed/valid.csv'), batch_size=32, shuffle=False)
test_loader = DataLoader(PhraseDataset('processed/test.csv'), batch_size=32, shuffle=False)

# Set up Model, Loss function and Optimizer

In [12]:
model = SimilarityModel(bert_pretrained)
loss = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
tokenizer = AutoTokenizer.from_pretrained(bert_pretrained)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Train Model

In [13]:
from transformer_class import train
train_logs, valid_logs, model = train(model, tokenizer, train_loader, val_loader, device, optimizer, loss, 3)

Epoch: 0, Average Train Loss: 0.04327274486422539
Average Epoch Valid Loss: 0.04954118654131889
Epoch: 1, Average Train Loss: 0.03755366429686546
Average Epoch Valid Loss: 0.044957973062992096
Epoch: 2, Average Train Loss: 0.033528756350278854
Average Epoch Valid Loss: 0.04191714897751808


In [16]:
import pickle
import os
version = 'v1.1_bert_base_SimilarityModel_MSE_Adam_1e-5_3epochs'
os.mkdir('models/'+version+'/logs')
torch.save(model.state_dict(), f'models/{version}/model.pth')
# save logs
with open(f'models/{version}/logs/train_logs.pkl', 'wb') as f:
    pickle.dump(train_logs, f)
with open(f'models/{version}/logs/valid_logs.pkl', 'wb') as f:
    pickle.dump(valid_logs, f)

# Evaluate Model